In [1]:
from datetime import date, timedelta
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm, tnrange

from sklearn.metrics import mean_squared_error
import lightgbm as lgb

import mlflow
import mlflow.sklearn

from config import (
    RAW_DATA_DIR,
    FEATURE_DIR,
    LAG_DICT,
    SLIDING_DICT
)

/Users/liuyu/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
# solve lightgbm error on MAC
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
# load data
df_train = pd.read_csv(
    RAW_DATA_DIR+'train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    RAW_DATA_DIR+'test.csv', usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    RAW_DATA_DIR+'items.csv',
).set_index("item_nbr")

stores = pd.read_csv(
    RAW_DATA_DIR+'stores.csv',
).set_index("store_nbr")

### Test Period

2017-08-16 to 2017-08-31

In [4]:
test_start = date(2017, 8, 16)
test_end = date(2017,8, 31)

In [5]:
valid_start = test_start - timedelta(16)
while(1):
    if valid_start.weekday() == test_start.weekday():
        break
    valid_start = valid_start-timedelta(days=1)
valid_end = valid_start + timedelta(15)
print('valid starts from {} to {}'.format(valid_start, valid_end))

valid starts from 2017-07-26 to 2017-08-10


### Valid Period

Considering the more nearer peiods of sales data may have more in common, it would be better to find the nearest period as valid period.

Based on the analysis before, we assume the sales data is periodically with the frequency of 7 days, so we want to keep that feature same
in the train, valid and test period.

So finally, we choose valid period:

2017-07-26 to 2017-08-10


In [6]:
valid_start = date(2017, 7, 26)
valid_end = date(2017, 8, 10)

### Filter Period

#### Earthquake happended on April 16, 2016. It may affect for the next several weeks.

In [7]:
filter_date = date(2016,4,16) + timedelta(7*4)
lag_max = 140
train_start=  filter_date+timedelta(days=lag_max)

while(1):
    train_start = train_start + timedelta(1)
    if train_start.weekday() == valid_start.weekday():
        break
print('train datasets starts from {}'.format(train_start))

train datasets starts from 2016-10-05


### Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.


In [8]:
df_train = df_train[df_train['date']>=filter_date]

/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


#### Promo feature

In [10]:
promo_train = df_train.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]]

# missing onpromotions filling
promo_train = promo_train.unstack(level=-1).fillna(False)
promo_train.columns = promo_train.columns.get_level_values(1)

In [11]:
# missing test onpromotions filling
promo_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_test.columns = promo_test.columns.get_level_values(1)
# filter those items/stores in promo_test but not in promo_train
promo_test = promo_test.reindex(promo_train.index).fillna(False)

In [12]:
promo_features = pd.concat([promo_train, promo_test], axis=1)
del promo_test, promo_train

## Label

In [15]:
# label
df_train = df_train.set_index(["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(level=-1).fillna(0)

### Item

In [16]:
items = items.reindex(df_train.index.get_level_values(1))

#### Item Family

In [75]:
items['family'] = items['family'].astype('category')
item_family_features = items.family.cat.codes.values

#### Item's class

In [77]:
items['class'] = items['class'].astype('category')
item_class_features = items['class'].cat.codes.values

### Store

In [40]:
stores = stores.reindex(df_train.index.get_level_values(0))

#### Store's city

In [78]:
stores['city'] = stores['city'].astype('category')
store_city_features = stores['city'].cat.codes.values

#### Store's state

In [79]:
stores['state'] = stores['state'].astype('category')
store_state_features = stores['state'].cat.codes.values

#### Store's type

In [80]:
stores['type'] = stores['type'].astype('category')
store_type_features = stores['type'].cat.codes.values

#### Store's cluster

In [81]:
stores['cluster'] = stores['cluster'].astype('category')
store_cluster_features = stores['cluster'].cat.codes.values

In [ ]:
df_train.columns = df_train.columns.get_level_values(1)

#### Filling missing date

In [25]:
date_list = df_train.columns
obj_list = pd.date_range(filter_date, test_start-timedelta(1))
diff_list = list(set(obj_list) - set(date_list)) 
for i in diff_list:
    print(i)
    df_train[i] = 0

2016-12-25 00:00:00


In [26]:
date_list = promo_features.columns
obj_list = pd.date_range(filter_date, test_end)
diff_list = list(set(obj_list) - set(date_list)) 
for i in diff_list:
    print(i)
    promo_features[i] = 0

2016-12-25 00:00:00


#### Lagging and sliding windows

In [27]:
LAG_DICT = {'unit_sales': [1,2,3,4,5,6,7,14,21,28,35,42,49,56,63],
            'onpromotion': [2, 3,4,5,6, 7, 14, 21]}

SLIDING_DICT = {'unit_sales': [3, 4, 5, 6, 7, 14, 21, 30, 60, 63]}

# initialise dirs
RAW_DATA_DIR = 'datasets/'

In [82]:
def get_timespan(df, 
                 start_time,
                 minus,
                 periods,
                 freq='D'):
    return df[pd.date_range(start_time - timedelta(days=minus), periods=periods, freq=freq)]

def gen_dataset(df, 
                promo_features,
                item_family_features,
                item_class_features,
                store_city_features,
                store_state_features,
                store_type_features,
                store_cluster_features,
                start_time,
                is_train=True):
    # init
    X = pd.DataFrame()
    
    for i in LAG_DICT['unit_sales']:
        X['lag_{}_sales'.format(i)] = get_timespan(df, start_time, i, 1).values.ravel()
    
    for i in LAG_DICT['onpromotion']:
        X['sum_{}_promo'.format(i)] = get_timespan(promo_features, start_time, i, 1).sum(axis=1).ravel()

    for i in SLIDING_DICT['unit_sales']:
        X["mean_{}_sales".format(i)] = get_timespan(df, start_time, i, i).mean(axis=1).values
        X["std_{}_sales".format(i)] = get_timespan(df, start_time, i, i).std(axis=1).values

    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, start_time, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, start_time, 140-i, 20, freq='7D').mean(axis=1).values
        
    # for the next to-predict 16 days 
    for i in range(16):
        X["promo_{}".format(i)] = promo_features[start_time + timedelta(days=i)].values.astype(np.uint8)

    X['item_family_features'] = item_family_features

    X['item_class_features'] = item_class_features

    X['store_city_features'] = store_city_features

    X['store_state_features'] = store_state_features

    X['store_type_features'] = store_type_features

    X['store_cluster_features'] = store_cluster_features
        
    if is_train:
        y = df[pd.date_range(start_time, periods=16)].values
        return X, y
    return X


#### Generate train, valid and test sets

In [83]:
print("Preparing dataset...")

nbr_weeks = int((valid_start - train_start).days/7)

X_l, y_l = [], []

for i in tqdm(range(nbr_weeks), desc = 'No. of week'):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = gen_dataset(
        df_train,
        promo_features,
        item_family_features,
        item_class_features,
        store_city_features,
        store_state_features,
        store_type_features,
        store_cluster_features,
        train_start + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
#     break


No. of week:   0%|          | 0/42 [00:00<?, ?it/s]

Preparing dataset...



No. of week: 100%|██████████| 42/42 [00:34<00:00,  1.30it/s]

In [84]:
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

In [85]:

X_val, y_val = gen_dataset(df_train,
                           promo_features,
                           item_family_features,
                           item_class_features,
                           store_city_features,
                           store_state_features,
                           store_type_features,
                           store_cluster_features,
                           valid_start)
X_test = gen_dataset(df_train, 
                    promo_features,
                    item_family_features,
                    item_class_features,
                    store_city_features,
                    store_state_features,
                    store_type_features,
                    store_cluster_features,
                    test_start, is_train=False)

#### Train Model

In [86]:
print("Training and predicting models...")
params = {
    'num_leaves': 2**5 - 1,
    'objective': 'regression_l2',
    'max_depth': 8,
    'min_data_in_leaf': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 200
val_pred = []
test_pred = []
cate_vars = ['item_family_features',
            'item_class_features',
            'store_city_features',
            'store_state_features',
            'store_type_features',
            'store_cluster_features']

Training and predicting models...


In [87]:
for i in tqdm(range(16)):
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * nbr_weeks) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)

    bst = lgb.train(
        params,
        dtrain,
        num_boost_round=MAX_ROUNDS,
#         verbose_eval = False,
        valid_sets=[dtrain, dval], early_stopping_rounds=50)
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))
    



  0%|          | 0/16 [00:00<?, ?it/s]/Users/liuyu/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	training's l2: 1.0669	valid_1's l2: 1.00319
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.997115	valid_1's l2: 0.936875
[3]	training's l2: 0.93291	valid_1's l2: 0.8756
[4]	training's l2: 0.875641	valid_1's l2: 0.821276
[5]	training's l2: 0.82264	valid_1's l2: 0.77096
[6]	training's l2: 0.774825	valid_1's l2: 0.725402
[7]	training's l2: 0.731454	valid_1's l2: 0.684427
[8]	training's l2: 0.692291	valid_1's l2: 0.647398
[9]	training's l2: 0.656942	valid_1's l2: 0.614013
[10]	training's l2: 0.624789	valid_1's l2: 0.58376
[11]	training's l2: 0.595655	valid_1's l2: 0.55643
[12]	training's l2: 0.569987	valid_1's l2: 0.532496
[13]	training's l2: 0.546698	valid_1's l2: 0.510864
[14]	training's l2: 0.524965	valid_1's l2: 0.49061
[15]	training's l2: 0.505157	valid_1's l2: 0.472122
[16]	training's l2: 0.487249	valid_1's l2: 0.455509
[17]	training's l2: 0.471025	valid_1's l2: 0.440529
[18]	training's l2: 0.45627	valid_1's l2: 0.426768
[19]	training's l2: 0.4


  6%|▋         | 1/16 [03:01<45:21, 181.41s/it]

[1]	training's l2: 0.963985	valid_1's l2: 0.924324
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.904899	valid_1's l2: 0.867583
[3]	training's l2: 0.851413	valid_1's l2: 0.816018
[4]	training's l2: 0.802821	valid_1's l2: 0.769379
[5]	training's l2: 0.758868	valid_1's l2: 0.727085
[6]	training's l2: 0.719869	valid_1's l2: 0.689843
[7]	training's l2: 0.683853	valid_1's l2: 0.655212
[8]	training's l2: 0.651996	valid_1's l2: 0.62482
[9]	training's l2: 0.622609	valid_1's l2: 0.596642
[10]	training's l2: 0.595832	valid_1's l2: 0.571112
[11]	training's l2: 0.571984	valid_1's l2: 0.548512
[12]	training's l2: 0.550006	valid_1's l2: 0.52748
[13]	training's l2: 0.530099	valid_1's l2: 0.508501
[14]	training's l2: 0.512468	valid_1's l2: 0.49201
[15]	training's l2: 0.496054	valid_1's l2: 0.476464
[16]	training's l2: 0.481223	valid_1's l2: 0.462484
[17]	training's l2: 0.467852	valid_1's l2: 0.449908
[18]	training's l2: 0.455661	valid_1's l2: 0.438444
[19]	training'


 12%|█▎        | 2/16 [05:56<41:52, 179.45s/it]

...


 88%|████████▊ | 14/16 [40:18<05:37, 168.66s/it]

[1]	training's l2: 1.07583	valid_1's l2: 1.0079
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 1.01041	valid_1's l2: 0.945488
[3]	training's l2: 0.95114	valid_1's l2: 0.888773
[4]	training's l2: 0.89986	valid_1's l2: 0.839439
[5]	training's l2: 0.853454	valid_1's l2: 0.794972
[6]	training's l2: 0.80896	valid_1's l2: 0.752569
[7]	training's l2: 0.768806	valid_1's l2: 0.714672
[8]	training's l2: 0.732376	valid_1's l2: 0.679898
[9]	training's l2: 0.701204	valid_1's l2: 0.650276
[10]	training's l2: 0.671246	valid_1's l2: 0.621866
[11]	training's l2: 0.645545	valid_1's l2: 0.597639
[12]	training's l2: 0.622346	valid_1's l2: 0.575651
[13]	training's l2: 0.601335	valid_1's l2: 0.555841
[14]	training's l2: 0.580557	valid_1's l2: 0.536427
[15]	training's l2: 0.561837	valid_1's l2: 0.518929
[16]	training's l2: 0.544773	valid_1's l2: 0.503201
[17]	training's l2: 0.529382	valid_1's l2: 0.488836
[18]	training's l2: 0.516501	valid_1's l2: 0.47688
[19]	training's l2:


 94%|█████████▍| 15/16 [43:07<02:48, 168.54s/it]

[1]	training's l2: 0.973227	valid_1's l2: 0.938425
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.918028	valid_1's l2: 0.884992
[3]	training's l2: 0.868152	valid_1's l2: 0.836293
[4]	training's l2: 0.823111	valid_1's l2: 0.792309
[5]	training's l2: 0.782324	valid_1's l2: 0.752467
[6]	training's l2: 0.74538	valid_1's l2: 0.716414
[7]	training's l2: 0.711998	valid_1's l2: 0.683923
[8]	training's l2: 0.681776	valid_1's l2: 0.65449
[9]	training's l2: 0.65538	valid_1's l2: 0.628725
[10]	training's l2: 0.630533	valid_1's l2: 0.604585
[11]	training's l2: 0.608015	valid_1's l2: 0.582848
[12]	training's l2: 0.58769	valid_1's l2: 0.56306
[13]	training's l2: 0.569309	valid_1's l2: 0.545149
[14]	training's l2: 0.552576	valid_1's l2: 0.528897
[15]	training's l2: 0.537457	valid_1's l2: 0.514256
[16]	training's l2: 0.523696	valid_1's l2: 0.501021
[17]	training's l2: 0.511242	valid_1's l2: 0.489024
[18]	training's l2: 0.49993	valid_1's l2: 0.478099
[19]	training's l


100%|██████████| 16/16 [45:55<00:00, 168.34s/it]

#### Generate submission

In [95]:
mse = mean_squared_error(y_val, np.array(val_pred).transpose())

mlflow.set_experiment('grocery forecasting')
with mlflow.start_run(run_name='lgbm'):
    mlflow.log_param('model', 'lgbm')
    mlflow.log_param('train starts', train_start)
    mlflow.log_params(params)
    mlflow.log_param('lagging', LAG_DICT.values())
    mlflow.log_param('slidingWindows', SLIDING_DICT.values())
    mlflow.log_param('item_info', 'Yes')
    mlflow.log_param('store_info', 'Yes')
    mlflow.log_param('private score', 0.52193)
    mlflow.log_param('private rank', '14%')
    mlflow.log_param('public score', 0.51609)

    mlflow.log_metric('mse', mse)
    
print("Validation mse:", mse)

Validation mse: 0.35123858092295934


In [88]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_train.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb.csv', float_format='%.4f', index=None)

Making submission...
